# DataLoaderの使い方

## 目的：Flix DataLodarを使ってどうやって画像をロードして，処理するかを学ぶ．

In [1]:
using Pkg
using MLDatasets: MNIST
using Flux.Data: DataLoader
using Flux: onehotbatch

In [2]:
Pkg.status()

      Status `~/Dropbox/01code/julia/intro_julia/Flux/Project.toml`
  [052768ef] CUDA v3.3.5 ⚲
  [587475ba] Flux v0.12.6 ⚲
  [1902f260] Knet v1.4.8 ⚲
  [eb30cadb] MLDatasets v0.5.9 ⚲
  [91a5bcdd] Plots v1.20.0 ⚲
  [c3e4b0f8] Pluto v0.15.1 ⚲
  [92933f4c] ProgressMeter v1.7.1


In [3]:
VERSION

v"1.6.0"

## 1. MNISTデータセットを読み込む
- `train_x`は28*28**60000の多次元次元配列で，60000の要素それぞれが28*28の配列を持っている．
    - それぞれの配列は28*28の配列で，グレースケールの手書き画像を表現している．
    - 28*28の配列のそれぞれの要素はピクセルで，輝度の値を持っている．
- `train_y`は60000要素のベクトルで，それぞれの要素はラベルを格納している（０から９のラベル）．

In [4]:
train_x, train_y = MNIST.traindata(Float32)
test_x, test_y = MNIST.testdata(Float32)
@show size(train_x)
@show size(train_y)
@show size(test_x)
@show size(test_y)

size(train_x) = (28, 28, 60000)
size(train_y) = (60000,)
size(test_x) = (28, 28, 10000)
size(test_y) = (10000,)


(10000,)

## 2. DataLoaderにデータセットを読み込む
ここではまず，Fluxが読み取れるように学習・テストデータの配列のサイズを整形する．
どのように整形するかというと，
- モデルのインプットとアウトプットの次元と合うように学習データのサイズを合わせる

例えば，もし扱うモデルの入力層が28*28*1の配列を受け取るとしたら，以下のように学習・テストデータの配列をrespaheする：

In [5]:
train_x = reshape(train_x, 28, 28, 1, :)
test_x = reshape(test_x, 28, 28, 1, :)
@show size(train_x)
@show size(test_x)

size(train_x) = (28, 28, 1, 60000)
size(test_x) = (28, 28, 1, 10000)


(28, 28, 1, 10000)

次に，`onehotbatch`をつかて，ラベルを格納したベクトルをエンコードする．
その際，データセットのカテゴリの数と同じ次元のベクトルでなければダメ．
`onehotbatch`はone-hot vectorのバッチ（行列）を作成する関数である：

- https://fluxml.ai/Flux.jl/stable/data/onehot/#Batches-1）
- https://fluxml.ai/Flux.jl/stable/data/onehot/

In [8]:
train_y = onehotbatch(train_y, 0:9)
test_y = onehotbatch(test_y, 0:9)
@show size(train_y)
@show size(test_y)

size(train_y) = (10, 10, 60000)
size(test_y) = (10, 10, 10000)


(10, 10, 10000)

ここで学習データとそのラベルを`DataLoader`に格納する：


In [11]:
data_loader = DataLoader(train_x, train_y, batchsize=128, shuffle=true)

LoadError: MethodError: no method matching DataLoader(::Array{Float32, 4}, ::Flux.OneHotArray{UInt32, 10, 2, 3, Matrix{UInt32}}; batchsize=1, shuffle=true)
[0mClosest candidates are:
[0m  DataLoader(::Any; batchsize, shuffle, partial, rng) at /Users/siida/.julia/packages/Flux/Zz9RI/src/data/dataloader.jl:77
[0m  DataLoader(::D, [91m::Int64[39m, [91m::Int64[39m, [91m::Bool[39m, [91m::Int64[39m, [91m::Vector{Int64}[39m, [91m::Bool[39m, [91m::R[39m) where {D, R<:Random.AbstractRNG} at /Users/siida/.julia/packages/Flux/Zz9RI/src/data/dataloader.jl:5[91m got unsupported keyword arguments "batchsize", "shuffle"[39m

## 3. データを繰り返す

In [14]:
#for (x, y) in data_loader
#   @assert size(x) == (28, 28, 1, 128) || size(x) == (28, 28, 1, 96)
#   @assert size(y) == (10, 128) || size(y) == (10, 96)
#   ...
#end 